# Настройки контекста

In [1]:
#Импорты тут
import os
import sys
from flask import Flask
import pandas as pd
import warnings
import json

In [2]:
spark_home = '/lib/spark2'

os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_DRIVER_PYTHON'] = '/bin/python'
os.environ['PYSPARK_PYTHON'] = '/bin/python'
os.environ['LD_LIBRARY_PATH'] = '/jupyter/lib'
 
sys.path.insert(0, os.path.join (spark_home,'python'))
 
sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkContext, SparkConf, HiveContext
 
conf = SparkConf().setAppName('Web Monitor Data Collector')\
    .setMaster("yarn-client")\
    .set('spark.dynamicAllocation.enabled', 'true')\
    .set('spark.executor.cores', '8')\
    .set('spark.executor.instances', '2')\
    .set('spark.executor.memory','8g')\
    .set('spark.driver.memory','8g')\
    .set('spark.driver.maxResultSize','6g')\
    .set('spark.shuffle.service.enabled','True')

In [3]:
from data_preparator.data_provider import DataProvider

# Контекст и сборщик данных

In [4]:
#запуск sparkcontext
sc = SparkContext.getOrCreate(conf=conf)

#Создаём инстанс провайдера данных
provider = DataProvider(sc)

# Запускаем веб сервис

In [5]:
from jinja2 import evalcontextfilter
#from flask_autodoc import Autodoc
from flask import jsonify, send_from_directory, request
import re
from markupsafe import Markup, escape
from functools import wraps

In [ ]:
#Создание Веб приложения
app = Flask("web_monitoring_back")
# Создание автодокера
#auto = Autodoc()
data_location = "data_preparator/data/"

# region Служебные методы
#обход бага в автодокере
_paragraph_re = re.compile(r'(?:\r\n|\r|\n){2,}')
@app.template_filter()
@evalcontextfilter
def nl2br(eval_ctx, value):
    result = u'\n\n'.join(u'<p>%s</p>' % p.replace('\n', '<br>\n')\
                        for p in _paragraph_re.split(escape(value)))
    if eval_ctx.autoescape:
        result = Markup(result)
    return result

def updatable(method):
    @wraps(method)
    def wrap(*args, **kwargs):
        if "update" in request.args:
            return method(*args, update=True, **kwargs)
        else:
            return method(*args, **kwargs)
    return wrap
# endregion

#region Информационные и технические методы
@app.route("/api")
#@auto.doc("info")
def documentation2():
    return "api"
    return auto.html()

@app.route("/api/version")
#@auto.doc("main")
def version2():
    """Возвращает версию API"""
    return "0.0.1"

@app.route("/api/clear_cache")
#@auto.doc("main")
def clear_cache():
    """Очищает кэшированные на сервере данные. При последующих обращениях будут осуществляться запросы к базе"""
    provider.clear_cache()
    return "Кэш очищен"

# endregion

# region risk/portfolio
@app.route("/api/risk/portfolio/metrics")
#@auto.doc("api/risk")
@updatable
def risk_portfolio_metrics(update=False):
    """Возвращает метрики риска портфеля: PD, LGD, LGDID, EL, COR"""
    data = provider.get_risk_portfolio_metrics(update)
    return jsonify(data)

@app.route("/api/risk/portfolio/products")
#@auto.doc("api/risk")
@updatable
def risk_portfolio_products(update=False):
    """Возвращает метрики риска портфеля: PD, LGD, LGDID, EL, COR в разрезе типов продуктов"""
    data = provider.get_risk_portfolio_products(update)
    return jsonify(data)
# endregion

# region risk/issues

# endregion

# region portfolio
@app.route("/api/portfolio/balances")
#@auto.doc("api/portfolio")
@updatable
def portfolio_balances(update=False):
    """Возвращает информацию об остатках по портфелю"""
    data = provider.get_portfolio_balances(update)
    return jsonify(data)

@app.route("/api/portfolio/products")
#@auto.doc("api/portfolio")
@updatable
def portfolio_products(update=False):
    """Возвращает информацию о прродуктовом составе портфеля в рублях и штуках"""
    data = provider.get_portfolio_products(update)
    return jsonify(data)

@app.route("/api/portfolio/npl")
#@auto.doc("api/portfolio")
@updatable
def portfolio_npl(update=False):
    """Возвращает информацию о npl портфеля"""
    data = provider.get_portfolio_npl(update)
    return jsonify(data)

@app.route("/api/portfolio/dr")
#@auto.doc("api/portfolio")
@updatable
def portfolio_dr(update=False):
    """Возвращает информацию о dr портфеля"""
    data = provider.get_portfolio_dr(update)
    return jsonify(data)

# endregion


# region DataQuality Assesment
@app.route("/api/dqa/portfolio")
#@auto.doc("api/dqa")
@updatable
def dqa_portfolio(update=False):
    """Возвращает результаты тестов качества данных портфеля"""
    data = provider.get_dqa_portfolio(update)
    return jsonify(data)

@app.route("/api/dqa/issues")
#@auto.doc("api/dqa")
@updatable
def dqa_issues(update=False):
    """Возвращает результаты тестов качества данных по выдачам"""
    data = provider.get_dqa_issues(update)
    return jsonify(data)

@app.route("/api/dqa/comparisson")
#@auto.doc("api/dqa")
@updatable
def dqa_comparisson(update=False):
    """Возвращает сравнение предыдущего и текущего состояния данных портфеля"""
    data = provider.get_dqa_comparison(update)
    return jsonify(data)


# endregion

@app.route("/")
#@auto.doc("")
def main():
    return "Главная страница бэкэнд сервиса веб мониторинга. По адресу /api есть описание доступных методов"


def read_config():
    import os
    final_config = {"host":"0.0.0.0",
        "port":5001,
        "debug": True                   
        }
    if os.path.exists("back_config.cfg"):
        import json
        with open("back_config.cfg", "r") as f:
            config_txt = f.read()
        config = json.loads(config_txt)
        if config["debug"]:
            port = config["dev_port"]
        else:
            port = config["prom_port"]
        final_config = {"host": config["host"],
                   "port": port,
                    "debug": config["debug"]
                   }

    print("Configuration:", final_config)
    return  final_config
 
        

cur_config = read_config()
app.run(host=cur_config["host"], port=cur_config["port"])#, debug=True)